In [1]:
import os
import datetime as dt
import numpy as np
from numpy import newaxis
import pandas as pd
import tensorflow as tf
!pip install geohash2
import geohash2 as gh
from math import floor
from tqdm import tqdm

  Stored in directory: /root/.cache/pip/wheels/69/63/0d/560a1741fa3f0ab897105cddb33f21f38c3330e4a57ea75db6
Successfully built geohash2


In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [3]:
linkts ='https://drive.google.com/open?id=1uJZxoU_TlhJo1ivP0T6ikH5kqORGnnrD'
linktr = "https://drive.google.com/open?id=1oqmqxb72zPnDLer2UqqRaAbCVggQPOB7"
fluff, tr_link = linktr.split('=')
fluff,ts_link = linkts.split('=')
downloaded_tr = drive.CreateFile({'id':tr_link}) 
downloaded_tr.GetContentFile('training.csv')  
downloaded_ts = drive.CreateFile({'id':ts_link}) 
downloaded_ts.GetContentFile('test.csv')  

W0617 14:34:44.471776 140675626592128 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNo

# Traffic Demand Prediction
</br>Dear Judge, I have written the notebook with the concept of walkthrough that serves as a journey how I approach the problem. This competition has been interesting and I have learned alot. Below is my journey to slove the problem.

</br>The Initial analysis of the traffic demand prediction is to understand the excel file given (training data). Excel csv file are given in the format of *geohash6*, *day*, *time* and *demand*. I proceed to check what are the maximum value of given column within the excel file. There are few things that I have incurred from the given training.csv file :


1.   The maximum and the mininum value in the column day is between 1 to 61. The timestamp of the data given are in every 15 minutes. This gives me around 5856 recording points to work with. Number of timestamp given in Time Series data is very important for Data Scientist to think how to work with given data in predictive modelling.

2.   With known unique value within **day** and **timestamp** column, I have found out that there's overlapping data in terms of each given timestamp, the only difference for data with same timestep is the demand and the geohash. This prompt me to model the problem of traffic prediction into different prespective.

3.   The geohash6 column represent information of latitude and longitude that is encoded using a geohash. Location information can be easily retrieve by using geohash2 library in python to decode and obtain relevant information.

This notebook has assumed that test data are given in the same format as training data where the time and day are not sorted out and location information given are only in geohash and normalized demand.


## WARNING!!! 
Before proceed with the notebook, please make sure that the 'training.csv' and 'test.csv' are in the same folder as this notebook.


In [0]:
import os
import datetime as dt
import numpy as np
from numpy import newaxis
import pandas as pd
import tensorflow as tf
import geohash2 as gh
from math import floor
from tqdm import tqdm

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.utils import *
from keras.utils.generic_utils import *

I have started by importing all the libraries I need to process and train a predictive model. Tester of the notebook might need to install geohash2 by using 'pip install geohash2' and install keras by using 'pip install keras'

In [0]:
def decode_csv(csvfile):
  """
  INPUT : csvfile
  OUTPUT : sorted csvfile based on time and day with information location decoded
  FUNCTION : This function take in csvfile and decode the geohash location, 
              add additional 2 column of 'latitude' and 'longitude' to the existing csvfile 
              and sorted the csv file based on the day and then the time columns
  """
  if not(os.path.isfile('{}_transform.csv'.format(csvfile.split('.')[0]))):
      df_total = pd.read_csv(csvfile)
      coordinate_list = df_total.apply(lambda x:gh.decode_exactly(x.geohash6), axis=1).tolist()
      df_total['latitude']= [x[0] for x in coordinate_list]
      df_total['longitude']= [x[1] for x in coordinate_list]
      df_total['time'] = df_total.apply(lambda x:dt.datetime.strptime(x.timestamp, '%H:%M').time(), axis=1)
      df_total = df_total.sort_values(by=['day','time','latitude','longitude'])
      n_days = len(df_total['day'].unique().tolist())
      timestamps = df_total.sort_values(by=['time']).time.unique().tolist()
      timestamps_dict = dict()
      for ii in range(len(timestamps)):
          timestamps_dict[timestamps[ii]] = ii+1
      df_total['timestep'] = df_total.apply(lambda x: timestamps_dict[x.time] + ((x.day-1)*96), axis=1 )
      df_total = df_total.drop(columns=['timestamp','geohash6'])
      df_total.to_csv('{}_transform.csv'.format(csvfile.split('.')[0]),index=False)
  else:
      df_total = pd.read_csv('{}_transform.csv'.format(csvfile.split('.')[0]))
  return df_total

df_train = decode_csv('training.csv')
df_test = decode_csv('test.csv')

First I have created the function of decode_csv. This function uses geohash2 library to obtain the exact number of latitude and longitude value with their respective error rate. I added 2 column to the imported data frame and I have also added a **time** column by converting the timestamp column into timestep data type in order for me to sort based on **day** and **time**. The reason behind this  is because data in the original **timestamp** column are read as string datatype. The function also save the transformed data frame as file to minimize the time taken to reload and applying the function to original csvfile.

Below are the details of transformed dataframe.

In [13]:
print(df_train.info())
print("number of unique latitude - ",len(df_train['latitude'].unique().tolist()))
print("number of unique longitude - ",len(df_train['longitude'].unique().tolist()))
print("number of unique day - ",len(df_train['day'].unique().tolist()))
print("number of unique timestep - ",len(df_train['timestep'].unique().tolist()))
print(df_train.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4206321 entries, 0 to 4206320
Data columns (total 6 columns):
day          int64
demand       float64
latitude     float64
longitude    float64
time         object
timestep     int64
dtypes: float64(3), int64(2), object(1)
memory usage: 192.6+ MB
None
number of unique latitude -  46
number of unique longitude -  36
number of unique day -  61
number of unique timestep -  5847
                 day        demand  ...      time      timestep
count   4.206321e+06  4.206321e+06  ...   4206321  4.206321e+06
unique           NaN           NaN  ...        96           NaN
top              NaN           NaN  ...  09:00:00           NaN
freq             NaN           NaN  ...     57281           NaN
mean    3.145299e+01  1.050907e-01  ...       NaN  2.965249e+03
std     1.768278e+01  1.592655e-01  ...       NaN  1.697748e+03
min     1.000000e+00  3.092217e-09  ...       NaN  1.000000e+00
25%     1.600000e+01  1.867379e-02  ...       NaN  1.478000e

After I have obtained the transformed training and test csv file, I proceed to model the problem as a spatiol-temporal problem. Spatiol temporal modelling is simply a set of measurements (features) that posses distinct spatial information ( posses x and y information) that we can observed the changes along time ( time-axis). This specific problem can be apply not only to traffic demand forecasting but also applies to video frame prediction, precipitation nowcasting problem etc.

</br>To model the traffic demand forecasting problem into spatiotemporal problem, I have proceed to get unique value of each extracted latitude and longitude value. Then I sorted the list of latitude and longitude value and find the increment of each next value. I noticed that the latitude and longitude are separated equally.
The result is there's  46 unique latitude value and 36 unique longitude value.


In [14]:
lats = df_train.sort_values(by=['latitude']).latitude.unique()
lats_inc = lats[1:] - lats[:-1]
print(lats_inc)
logts = df_train.sort_values(by=['longitude']).longitude.unique()
logts_inc = logts[1:] - logts[:-1]
print(logts_inc)

[0.00549316 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316
 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316
 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316
 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316
 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316
 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316
 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316 0.00549316
 0.00549316 0.00549316 0.00549316]
[0.01098633 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633
 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633
 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633
 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633
 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633
 0.01098633 0.01098633 0.01098633 0.01098633 0.01098633]


Then, I created a function get_raw_SPT. This function serves to create a grid of demand data in the form of rectangular grid, based on the unique number of latitude and longitude. Any nodes within the grid that's has no information are assume to be zero. This 3-Dimensional array of data is named RAW_SpatiolTemporal(SPT). 

The first step to populate the grid with the demand is to map each latitudes value and longitudes value to integer number starting from 1 to max(latitude or longitude). Then I proceed to loop through each unique timestep to populate the grid from the latitude and longitude mapping that I have done.

In the end, I manage to extract a 3D array with the shape of  (timestep, latitude, longitude) of training data to work with.

In [15]:
def get_raw_SPT(df,typ):
  """
  INPUT : transformed dataframe with location decoded and sorted by timestamp
  OUTPUT : 3D array data containing demand data with spatial information (2D array) arranged along with the time
  FUNCTION : populate 2D array with demand data from transformed csvfile by mapping longitude and latitude to x and y axis of 2D array data
              and stacked them up along with what time they happened.
  """
  latitudes = df.sort_values(by=['latitude']).latitude.unique().tolist()
  latitudes_dict = dict()
  for ii in range(len(latitudes)):
      latitudes_dict[latitudes[ii]] = ii
  longitudes = df.sort_values(by=['longitude']).longitude.unique().tolist()
  longitudes_dict = dict()
  for ii in range(len(longitudes)):
      longitudes_dict[longitudes[ii]] = ii

  if not(os.path.isfile('RAW_{}_NPY.npy'.format(typ))):  
      data = []

      for ii in range(len(df.timestep.unique().tolist())):
          df_ts = df.loc[df['timestep'] == ii+1]
          X = [latitudes_dict[i] for i in df_ts['latitude'].tolist()]
          Y = [longitudes_dict[i] for i in df_ts['longitude'].tolist()]
          Z = df_ts['demand'].tolist()
          matrix = np.zeros([len(latitudes),len(longitudes)])
          matrix[X,Y] = Z
          matrix = matrix[newaxis,:,:]
          if len(data) == 0:
              data = matrix
          else:
              data = np.concatenate([data,matrix],axis=0)
      np.save('RAW_{}_NPY.npy'.format(typ),data)
  else:
      data = np.load('RAW_{}_NPY.npy'.format(typ))
  return data
      
train_npy = get_raw_SPT(df_train,'train')
test_npy = get_raw_SPT(df_test,'test')
print(train_npy.shape)
print(test_npy.shape)

(5847, 46, 36)
(1440, 46, 36)


Next, to generate data and target for deep learning training purposes, I decided to do time series feature extraction. There's several term I have used here:


1.   train_frame/frame : how many time steps to be taken into consideration in generating 1 data point.
2.   stride                        : number of time steps to move the train_frame to generate next data point.
3.   window_observation : number of time steps to be considered when perform time series feature extraction ( mean, standard deviation and variance)
4.   delta/ delta_delta : rate of change of demand and first derivative rate of change of demand.

In the previously extracted raw_SPT data array, demands at each time step are presented in 2D array form which preserve the spatial information and arranged along with time axis.
In this scenario, I have decided to use train_frame of 7 time steps to generate data point in order to predict the demand of 5 time steps ahead. </br>

The feature extraction of data point starts with getting the rate of change of demand and first derivative of rate of change. I extracted 2D array demand at time equals to t-1 and t-2, and continue to use 2D array demand at time = t to minus with 2D array demand time equals to t-1 to obtain the rate of change and use the same method to get the first derivative of rate of change. </br>
Next, to obtain statistical feature from applying aggregration function on certain window of observational ( moving statistical feature), parameters such as window_observation need to be set in order to determine how many times steps before to apply aggregation function to get the statistical feature. </br>
Finally, all of 6 2D array will be concatenated together to create a data within 1 time step with 6 feature.</br>

To complete the generation of data points for training, I proceed to do the feature extraction process mentioned above for 7 times for each time steps and concatenate them into 1 single data point with array shape of ( 7, 46, 36, 6) where 7 is for 7 timesteps, 46 and 36 are respective number for latitude and longitude and 6 is the demand and demand features that has been extracted.
Each training point is then concatenate into 1 big npy array to feed into deep learning training process.</br>

For target data extraction, it's more straightforward where 5 timestep 2D data are extracted because we want to predict what's demand like for next 5 timesteps.

In [0]:

def time_fv_extract(data,iteration,train_frame,stride,window_observation=3):
    """
    INPUT : 
        data = raw_spationtemporal numpy array from previous function
        iteration = i-th iteration of data point generation
        train_frame = how many timesteps to be considered in 1 data points
        stride = how many timesteps jump for generating another data point
        window_observation = amount timesteps to be considered to apply statiscal aggregation function.
    OUPUT :
        feature extracted from raw_spatiotemporal data
    FUNCTION : feature extraction for making data-target set
    """
    _seg = data[iteration*stride:iteration*stride+train_frame,:,:]
    _1seg = data[(iteration-1)*stride:(iteration-1)*stride+train_frame,:,:]
    _2seg = data[(iteration-2)*stride:(iteration-2)*stride+train_frame,:,:]
    _delta = _seg-_1seg
    _1delta = _1seg-_2seg
    _deltadelta = _delta-_1delta
    _stats_seg = []
    for xx in range(train_frame):
        _forStats =  data[iteration*stride+xx-window_observation+1:iteration*stride+xx+1,:,:]
        _mean = np.mean(_forStats,axis=0)
        _std = np.std(_forStats,axis=0)
        _var = np.var(_forStats,axis=0)
        _mean = _mean[:,:,newaxis]
        _std = _std[:,:,newaxis]
        _var = _var[:,:,newaxis]
        _stats = np.concatenate([_mean,_std,_var],axis=-1)
        _stats = _stats[newaxis,:,:,:]
        if len(_stats_seg) == 0:
            _stats_seg = _stats
        else:
            _stats_seg = np.concatenate([_stats_seg,_stats],axis=0)
    _seg = _seg[:,:,:,newaxis]
    _delta = _delta[:,:,:,newaxis]
    _deltadelta = _deltadelta[:,:,:,newaxis]
    _res = np.concatenate([_seg,_delta,_deltadelta,_stats_seg],axis=-1)
    return _res

def data_gen(data,typ,train_frame=1344,target_frame=5,stride=5,):
    """
    INPUT : 
        data = raw_spationtemporal numpy array from previous function
        typ = for naming, whether "train" or "test"
        train_frame = how many timesteps to be considered in 1 data points
        stride = how many timesteps jump for generating another data point
    OUTPUT : 
        data and target set used for deep learning model.
    FUNCTION : concatenated extracted feature into a data target set for deep learning model
    """
    if not(os.path.isfile("{}_data_frame-{}_stride-{}.npy".format(typ,train_frame,stride)) and os.path.isfile("{}_target_frame-{}_stride-{}.npy".format(typ,train_frame,stride))):
        dt_points  = floor((data.shape[0]-train_frame)/stride)-5
        _data = []
        _target = []
        print("Number of {} Sample Generated - ".format(typ),dt_points)
        for ii in tqdm(range(2,dt_points)):
            _train_segment = time_fv_extract(data,ii,train_frame,stride)
            _target_segment = data[ii*stride+train_frame:ii*stride+train_frame+target_frame,:,:]
            _train_segment = _train_segment[newaxis,:,:,:,:]
            _target_segment = _target_segment[newaxis,:,:,:]
            if len(_data) == 0:
                _data = _train_segment
                _target = _target_segment
            else:
                _data = np.concatenate([_data,_train_segment],axis=0)
                _target = np.concatenate([_target,_target_segment],axis=0)
        np.save("{}_data_frame-{}_stride-{}.npy".format(typ,train_frame,stride),_data)
        np.save("{}_target_frame-{}_stride-{}.npy".format(typ,train_frame,stride),_target)
    else:
        _data = np.load("{}_data_frame-{}_stride-{}.npy".format(typ,train_frame,stride))
        _target = np.load("{}_target_frame-{}_stride-{}.npy".format(typ,train_frame,stride))
    return _data,_target



In [10]:
train_data, train_target = data_gen(train_npy,'train',train_frame=7,target_frame=5,stride=1)
test_data,test_target = data_gen(test_npy,'test',train_frame=7,target_frame=5,stride=1)
print('Train Data set train - {} target - {}.'.format(train_data.shape,train_target.shape))
print('Test Data set train - {} target - {}.'.format(test_data.shape,test_target.shape))

  1%|          | 30/5833 [00:00<00:19, 292.45it/s]

Number of train Sample Generated -  5835


  2%|▏         | 23/1426 [00:00<00:06, 223.53it/s]

Number of test Sample Generated -  1428


100%|██████████| 1426/1426 [03:25<00:00,  3.49it/s]


Train Data set train - (5833, 7, 46, 36, 6) target - (5833, 5, 46, 36).
Test Data set train - (1426, 7, 46, 36, 6) target - (1426, 5, 46, 36).


Some specific parameters setting to be inserted within the model formation are done here.

In [0]:
TRAIN_FRAME = 7
N_CHANNEL = 6
LATITUDE = 46
LONGITUDE = 36
STRIDE = 1

get_tt_data function is created to load the npy dataset for deep learning model training

In [0]:
def get_tt_data(trainframe=1344,stride=5,folder='train'):
    """
    INPUT : 
        train_frame = how many timesteps to be considered in 1 data points
        stride =  how many timesteps jump for generating another data point
        folder = data type to be extracted ( training or test)
    OUTPUT :
        data and target set loaded as numpy array
    FUNCTION : To load generated dataset
    """
    tr_dt = np.load("{}_data_frame-{}_stride-{}.npy".format(folder,trainframe,stride))
    tg_dt = np.load("{}_target_frame-{}_stride-{}.npy".format(folder,trainframe,stride))
    print("{} data shape - {}".format(folder,tr_dt.shape))
    print("{} target shape - {}".format(folder,tg_dt.shape))
    return tr_dt, tg_dt

After frame the problem into a spatiotemporal problem, I have done several research on which deep learning model works well with predicting the problem. The model is based on ConvLSTM that has been explained here (https://medium.com/neuronio/an-introduction-to-convlstm-55c9025563a7). In summary, In addition of using sigmoid and tanh within LSTM cell, Convolutional operations are used. </br>![alt text](https://cdn-images-1.medium.com/max/1000/1*u8neecA4w6b_F1NgnyPP0Q.png)  </br>
I have drawn inspiration from [Shi et al](https://papers.nips.cc/paper/5955-convolutional-lstm-network-a-machine-learning-approach-for-precipitation-nowcasting.pdf) with their application of ConvLSTM on precipitation forecasting. With some modification, I have constucted the model to be able to train and predict precipitation rate for 5 time step ahead.

In [0]:
def convlstm_model3(train_frame,n_channel,n_latitude,n_longitude):
    """
    INPUT :
        train_frame = how many timesteps to be considered in 1 data points
        n_channel = how many features within 1 timestep is extracted
        n_latitude = unique number of latitude
        n_longitude = unique number of longitude
    OUTPUT :
        ConvLSTM model created based on the settings specified in the input
    FUNCTION : Create structure of ConvLSTM model to solve spatiotemporal problem
    """
    input_layer = Input(batch_shape=(None, train_frame, n_latitude,n_longitude, n_channel))

    x = ConvLSTM2D(64, (3,3), strides=(1,1), padding='same', return_sequences=True)(input_layer)

    x1 = ConvLSTM2D(16, (3,3), return_sequences=True, padding='same', name='x1_layer_1')(x)
    #x1 = ConvLSTM2D(64, (3,3), return_sequences=True, padding='same', name='x1_layer_2')(x1)
    #x1 = ConvLSTM2D(1, (3,3), padding='same', name='x1_layer')(x)
    x1 = ConvLSTM2D(1, (1,1), name='x1_layer_2')(x1)
    x1 = Reshape((1, n_latitude, n_longitude))(x1)

    x2 = ConvLSTM2D(16, (3,3), return_sequences=True, padding='same', name='x2_layer_1')(x)
    #x2 = ConvLSTM2D(64, (3,3), return_sequences=True, padding='same', name='x2_layer_2')(x2)
    #x2 = ConvLSTM2D(1, (3,3), padding='same', name='x2_layer')(x)
    x2 = ConvLSTM2D(1, (1,1), name='x2_layer_2')(x2)
    x2 = Reshape((1, n_latitude, n_longitude))(x2)
    
    x3 = ConvLSTM2D(16, (3,3), return_sequences=True, padding='same', name='x3_layer_1')(x)
    #x3 = ConvLSTM2D(64, (3,3), return_sequences=True, padding='same', name='x3_layer_2')(x3)
    #x3 = ConvLSTM2D(1, (3,3), padding='same', name='x3_layer')(x)
    x3 = ConvLSTM2D(1, (1,1), name='x3_layer_2')(x3)
    x3 = Reshape((1, n_latitude, n_longitude))(x3)
    
    x4 = ConvLSTM2D(16, (3,3), return_sequences=True, padding='same', name='x4_layer_1')(x)
    #x4 = ConvLSTM2D(64, (3,3), return_sequences=True, padding='same', name='x4_layer_2')(x4)
    #x4 = ConvLSTM2D(1, (3,3), padding='same', name='x4_layer')(x)
    x4 = ConvLSTM2D(1, (1,1), name='x4_layer_2')(x4)
    x4 = Reshape((1, n_latitude, n_longitude))(x4)
    
    x5 = ConvLSTM2D(16, (3,3), return_sequences=True, padding='same', name='x5_layer_1')(x)
    #x5 = ConvLSTM2D(64, (3,3), return_sequences=True, padding='same', name='x5_layer_2')(x5)
    #x5 = ConvLSTM2D(1, (3,3), padding='same', name='x5_layer')(x)
    x5 = ConvLSTM2D(1, (1,1), name='x5_layer_2')(x5)
    x5 = Reshape((1, n_latitude, n_longitude))(x5)
    

    output = Concatenate(name='concat_layer', axis=1)([x1, x2, x3, x4, x5])
    #output = Concatenate(name='concat_layer', axis=1)([x1, x2, x3])

    model = Model(inputs=input_layer, outputs=output)
    model.summary()
    return model

In [0]:
def train_model(model_name='ConvLSTM',opt='Adam'):
    """
    INPUT : optimizer function for deep learning model
    FUNCTION : wrapper function to start for training process
    """
    train, target = get_tt_data(TRAIN_FRAME,STRIDE)
    if model_name == "GoogleCongestion":
        model = ConvLSTM_GoogleCongestion(TRAIN_FRAME,N_CHANNEL,LATITUDE,LONGITUDE)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001), metrics=['acc'])
    elif model_name == 'ConvLSTM':
        model = convlstm_model3(TRAIN_FRAME,N_CHANNEL,LATITUDE,LONGITUDE)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001), metrics=['acc'])
    model.fit(train,target,batch_size=32, epochs=10, verbose=1, validation_split = 0.2)

    model.save("{}_TRIAL5_tf-{}_sd-{}_{}.h5".format(model_name,TRAIN_FRAME,STRIDE,opt))

Proceed to train deep learning model by calling the train_model function

In [16]:
train_model(model='ConvLSTM')

W0617 15:49:58.439812 140675626592128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0617 15:49:58.593121 140675626592128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0617 15:49:58.610487 140675626592128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



train data shape - (5833, 7, 46, 36, 6)
train target shape - (5833, 5, 46, 36)


W0617 15:50:00.951709 140675626592128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7, 46, 36, 6) 0                                            
__________________________________________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)     (None, 7, 46, 36, 64 161536      input_1[0][0]                    
__________________________________________________________________________________________________
x1_layer_1 (ConvLSTM2D)         (None, 7, 46, 36, 16 46144       conv_lst_m2d_1[0][0]             
__________________________________________________________________________________________________
x2_layer_1 (ConvLSTM2D)         (None, 7, 46, 36, 16 46144       conv_lst_m2d_1[0][0]             
__________________________________________________________________________________________________
x3_layer_1

W0617 15:50:01.734393 140675626592128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0617 15:50:07.696327 140675626592128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0617 15:50:08.064543 140675626592128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 4666 samples, validate on 1167 samples
Epoch 1/10
4666/4666 [==============================] - 137s 29ms/step - loss: 0.0022 - acc: 0.6060 - val_loss: 0.0014 - val_acc: 0.6430
Epoch 2/10
4666/4666 [==============================] - 127s 27ms/step - loss: 0.0011 - acc: 0.6476 - val_loss: 0.0012 - val_acc: 0.6629
Epoch 3/10
4666/4666 [==============================] - 127s 27ms/step - loss: 9.3665e-04 - acc: 0.6591 - val_loss: 0.0010 - val_acc: 0.6637
Epoch 4/10
4666/4666 [==============================] - 127s 27ms/step - loss: 9.0251e-04 - acc: 0.6640 - val_loss: 0.0010 - val_acc: 0.6751
Epoch 5/10
4666/4666 [==============================] - 127s 27ms/step - loss: 8.7530e-04 - acc: 0.6698 - val_loss: 0.0010 - val_acc: 0.6715
Epoch 6/10
4666/4666 [==============================] - 127s 27ms/step - loss: 8.5888e-04 - acc: 0.6707 - val_loss: 9.8717e-04 - val_acc: 0.6764
Epoch 7/10
4666/4666 [==============================] - 127s 27ms/step - loss: 8.5684e-04 - acc: 0.6737 - val_

Below is the metrics to see how good is the predictive ability of the model.

In [0]:
from sklearn.metrics import mean_squared_error

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def evaluate_model(model='ConvLSTM',opt='Adam'):
    """
    FUNCTION : to evaluate the model ability to predict using SMAPE
    """
    if model == 'ConvLSTM':
        model = load_model("{}_TRIAL5_tf-{}_sd-{}_{}.h5".format(model,TRAIN_FRAME,STRIDE,opt))
    elif model == 'GoogleCongestion':
        model = load_model("{}_TRIAL5_tf-{}_sd-{}_{}.h5".format(model,TRAIN_FRAME,STRIDE,opt))
    test_data, test_target = get_tt_data(TRAIN_FRAME,STRIDE,folder='test')
    pred = model.predict(test_data)
    pred = pred.reshape(pred.shape[0],pred.shape[1],pred.shape[2]*pred.shape[3])
    test_target = test_target.reshape(test_target.shape[0],test_target.shape[1],test_target.shape[2]*test_target.shape[3])
    smape_seg = 0
    rmse_seg = 0
    metrics = []
    five_metrics = []
    for ii in range(pred.shape[0]):
        five_smape = 0
        five_rmse = 0
        for jj in range(pred.shape[1]):
            pred_pertimestep = pred[ii,jj,:]
            test_targetpertimestep = test_target[ii,jj,:]
            smape_seg = smape(test_targetpertimestep,pred_pertimestep)
            five_smape = smape(test_targetpertimestep,pred_pertimestep) + five_smape
            rmse_seg = np.sqrt(mean_squared_error(test_targetpertimestep,pred_pertimestep))
            five_rmse = np.sqrt(mean_squared_error(test_targetpertimestep,pred_pertimestep)) + five_rmse
            metrics.append((smape_seg,rmse_seg))
        five_metrics.append((five_smape,five_rmse))
    five_metrics = np.asarray(five_metrics)
    metrics = np.asarray(metrics)
    print("Average Symmetric Mean Absolute Percentage Error of prediction is (per 5 timestep)",np.mean(five_metrics,axis=0)[0])
    print("Average Root Mean Squared Error of prediction is (per 5 timestep)",np.mean(five_metrics,axis=0)[1])
    
    print("Average Symmetric Mean Absolute Percentage Error of prediction is (per timestep)",np.mean(metrics,axis=0)[0])
    print("Average Root Mean Squared Error of prediction is (per timestep)",np.mean(metrics,axis=0)[1])


In [35]:
evaluate_model()

test data shape - (1426, 7, 46, 36, 6)
test target shape - (1426, 5, 46, 36)
Average Symmetric Mean Absolute Percentage Error of prediction is (per 5 timestep) 672.2149981587627
Average Root Mean Squared Error of prediction is (per 5 timestep) 0.14479674231288528
Average Symmetric Mean Absolute Percentage Error of prediction is (per timestep) 134.44299963175246
Average Root Mean Squared Error of prediction is (per timestep) 0.0289593484625771


In [0]:
def ConvLSTM_GoogleCongestion(train_frame,n_channel,n_latitude,n_longitude):
    input_layer = Input(batch_shape=(None, train_frame, n_latitude,n_longitude, n_channel))
    x = ConvLSTM2D(40, (3,3), strides=(1,1), padding='same', activation= 'relu', return_sequences=True, name='1_ConvLSTM')(input_layer)
    x = BatchNormalization()(x)
    x = ConvLSTM2D(40, (3,3), strides=(1,1), padding='same', activation= 'relu', return_sequences=True, name='2_ConvLSTM')(x)
    x = BatchNormalization()(x)
    x = ConvLSTM2D(40, (3,3), strides=(1,1), padding='same', activation= 'relu', return_sequences=True, name='3_ConvLSTM')(x)
    x = BatchNormalization()(x)
    x = ConvLSTM2D(40, (3,3), strides=(1,1), padding='same', activation= 'relu', return_sequences=True, name='4_ConvLSTM')(x)
    x = BatchNormalization()(x)
    x = Conv3D(5,(3,3,3),padding='same',data_format='channels_first', name='1_3DConv')(x)
    x = Conv3D(1,(3,3,3),padding='same',data_format='channels_last', name='2_3DConv')(x)
    output = Reshape((5,n_latitude,n_longitude)) (x)
    model = Model(inputs=input_layer, outputs=output)
    model.summary()
    return model

In [10]:
train_model(model='GoogleCongestion')

train data shape - (5833, 7, 46, 36, 6)
train target shape - (5833, 5, 46, 36)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 7, 46, 36, 6)      0         
_________________________________________________________________
1_ConvLSTM (ConvLSTM2D)      (None, 7, 46, 36, 40)     66400     
_________________________________________________________________
batch_normalization_5 (Batch (None, 7, 46, 36, 40)     160       
_________________________________________________________________
2_ConvLSTM (ConvLSTM2D)      (None, 7, 46, 36, 40)     115360    
_________________________________________________________________
batch_normalization_6 (Batch (None, 7, 46, 36, 40)     160       
_________________________________________________________________
3_ConvLSTM (ConvLSTM2D)      (None, 7, 46, 36, 40)     115360    
_______________________________________________________________